## Extract Data of Site Fundamentus

All credits for the api founded on https://github.com/phoemur/fundamentus/blob/033c5341086299f800f01519ccc857829a4800c6/fundamentus.py#L95

# 1.0 Libs And Functions

In [1]:
# data manipulation libs
import pandas as pd
import numpy as np
# date and time libs
import datetime as dt
from datetime import timedelta
# using a soup lib to scrapp the page
import urllib.request
import urllib.parse
# lib to passe cookies
import http.cookiejar
from lxml.html import fragment_fromstring
import re
# libs to clean data exported data
from collections import OrderedDict
from decimal import Decimal

In [2]:
# personalize class content all the functions i need
class functions(object):
    def format_currency(self, x):
        return "R${:,.2f}".format(x)

    def format_perc(self, x):
        return "{}%".format(x)

    def today(self):
        return dt.date.today()

    def replace_nan(self,df,column,to_replace,repl):
        df[column] = df[column].replace(to_replace,repl)

    def change_type(self,df,column,type):
        df[column] = df[column].astype(type)

    def options(self):
        global pd_options
        pd_options = pd.options.mode.chained_assignment = None

    def column_index(self, df, query_cols):
        cols = df.columns.values
        sidx = np.argsort(cols)
        return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

    def inicio_mes(self):
        hoje = dt.datetime.today() 
        inicio_mes_data = hoje - timedelta(hoje.day)+ timedelta(days=1)
        return inicio_mes_data
        
    def round_data(self,df,columns_to_round):
        df[columns_to_round] = np.round(df[columns_to_round],2)

    def todecimal(self,string):
        string = string.replace('.', '')
        string = string.replace(',', '.')
        
        if (string.endswith('%')):
            string = string[:-1]
            return Decimal(string) / 100
        else:
            return Decimal(string)


personal_functions = functions()

# 2.0 Extract Data

In [3]:
# url to extract'
fundamentus_url = r'https://www.fundamentus.com.br/resultado.php'

In [4]:
# get the url response
cookie_jar = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]

In [5]:
# create a dict
data = {'pl_min': '',
            'pl_max': '',
            'pvp_min': '',
            'pvp_max' : '',
            'psr_min': '',
            'psr_max': '',
            'divy_min': '',
            'divy_max': '',
            'pativos_min': '',
            'pativos_max': '',
            'pcapgiro_min': '',
            'pcapgiro_max': '',
            'pebit_min': '',
            'pebit_max': '',
            'fgrah_min': '',
            'fgrah_max': '',
            'firma_ebit_min': '',
            'firma_ebit_max': '',
            'margemebit_min': '',
            'margemebit_max': '',
            'margemliq_min': '',
            'margemliq_max': '',
            'liqcorr_min': '',
            'liqcorr_max': '',
            'roic_min': '',
            'roic_max': '',
            'roe_min': '',
            'roe_max': '',
            'liq_min': '',
            'liq_max': '',
            'patrim_min': '',
            'patrim_max': '',
            'divbruta_min': '',
            'divbruta_max': '',
            'tx_cresc_rec_min': '',
            'tx_cresc_rec_max': '',
            'setor': '',
            'negociada': 'ON',
            'ordem': '1',
            'x': '28',
            'y': '16'}

In [6]:
# disable cookies
html = opener.open(fundamentus_url, urllib.parse.urlencode(data).encode('UTF-8'))
html_content = html.read().decode('ISO-8859-1')

In [7]:
# get the pattern to search inside te html_content
pattern = re.compile('<table id="resultado".*</table>', re.DOTALL)

In [8]:
# soup the page and get the pure html
final_soup = re.findall(pattern,html_content)[0]

In [9]:
# formated
final_page = fragment_fromstring(final_soup)

In [10]:
# create a dict to put the data inside
final_dict = OrderedDict()

In [11]:
# fuction that return a dict with all elements from the html
def return_function():
    for rows in final_page.xpath('tbody')[0].findall("tr"):
        final_dict.update({rows.getchildren()[0][0].getchildren()[0].text:
        {'Cotacao': personal_functions.todecimal(rows.getchildren()[1].text),
        'P/L': personal_functions.todecimal(rows.getchildren()[2].text),
        'P/VP': personal_functions.todecimal(rows.getchildren()[3].text),
        'PSR': personal_functions.todecimal(rows.getchildren()[4].text),
        'DY': personal_functions.todecimal(rows.getchildren()[5].text),
        'P/Ativo': personal_functions.todecimal(rows.getchildren()[6].text),
        'P/Cap.Giro': personal_functions.todecimal(rows.getchildren()[7].text),
        'P/EBIT': personal_functions.todecimal(rows.getchildren()[8].text),
        'P/ACL': personal_functions.todecimal(rows.getchildren()[9].text),
        'EV/EBIT': personal_functions.todecimal(rows.getchildren()[10].text),
        'EV/EBITDA': personal_functions.todecimal(rows.getchildren()[11].text),
        'Mrg.Ebit': personal_functions.todecimal(rows.getchildren()[12].text),
        'Mrg.Liq.': personal_functions.todecimal(rows.getchildren()[13].text),
        'Liq.Corr.': personal_functions.todecimal(rows.getchildren()[14].text),
        'ROIC': personal_functions.todecimal(rows.getchildren()[15].text),
        'ROE': personal_functions.todecimal(rows.getchildren()[16].text),
        'Liq.2meses': personal_functions.todecimal(rows.getchildren()[17].text),
        'Pat.Liq': personal_functions.todecimal(rows.getchildren()[18].text),
        'Div.Brut/Pat.': personal_functions.todecimal(rows.getchildren()[19].text),
        'Cresc.5anos': personal_functions.todecimal(rows.getchildren()[20].text)}})
    return final_dict
final_dict = return_function()

In [12]:
final_dict

OrderedDict([('AESB3',
              {'Cotacao': Decimal('9.38'),
               'P/L': Decimal('-1387.24'),
               'P/VP': Decimal('1.35'),
               'PSR': Decimal('2.005'),
               'DY': Decimal('0.0094'),
               'P/Ativo': Decimal('0.374'),
               'P/Cap.Giro': Decimal('1.90'),
               'P/EBIT': Decimal('11.08'),
               'P/ACL': Decimal('-1.00'),
               'EV/EBIT': Decimal('19.79'),
               'EV/EBITDA': Decimal('10.02'),
               'Mrg.Ebit': Decimal('0.181'),
               'Mrg.Liq.': Decimal('0.0537'),
               'Liq.Corr.': Decimal('3.63'),
               'ROIC': Decimal('0.0448'),
               'ROE': Decimal('-0.001'),
               'Liq.2meses': Decimal('20667900.00'),
               'Pat.Liq': Decimal('4175170000.00'),
               'Div.Brut/Pat.': Decimal('1.89'),
               'Cresc.5anos': Decimal('1.3024')}),
             ('ONCO3',
              {'Cotacao': Decimal('6.12'),
               '

In [39]:
final_df = pd.DataFrame(final_dict).transpose()

In [40]:
final_df.reset_index().rename(columns={'index':'COD_ACAO'},inplace=True)

In [41]:
final_df

,Cotacao,P/L,P/VP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/ACL,EV/EBIT,EV/EBITDA,Mrg.Ebit,Mrg.Liq.,Liq.Corr.,ROIC,ROE,Liq.2meses,Pat.Liq,Div.Brut/Pat.,Cresc.5anos
AESB3,9.38,-1387.24,1.35,2.005,0.0094,0.374,1.90,11.08,-1.00,19.79,10.02,0.181,0.0537,3.63,0.0448,-0.001,20667900.00,4175170000.00,1.89,1.3024
ONCO3,6.12,-318.86,1.51,0.838,0.00,0.419,4.95,8.83,-1.16,15.75,10.60,0.0949,0.0109,1.42,0.0557,-0.0047,8085020.00,2014190000.00,1.48,0.00
RAIL3,16.91,-282.74,2.08,3.433,0.0011,0.689,6.82,14.18,-1.56,18.79,9.95,0.2421,-0.0124,1.83,0.0577,-0.0074,268912000.00,15061000000.00,1.11,0.077
DMMO3,1.86,-258.31,-1.71,2.851,0.00,3.341,11.89,6.36,-1.37,5.89,5.41,0.4481,-0.011,2.25,0.7039,0.0066,17338800.00,-553275000.00,0.00,-0.1737
ELMD3,9.70,-165.61,1.84,1.941,0.00,0.680,-6.77,15.60,-1.80,21.78,9.95,0.1244,0.0024,0.64,0.0504,-0.0111,3058160.00,739445000.00,0.91,1.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NGRD3,1.42,254.81,0.75,1.288,0.0105,0.472,1.68,36.34,4.78,15.39,3.88,0.0354,0.0031,2.48,0.0219,0.0029,1218430.00,451675000.00,0.16,0.1122
LJQQ3,4.20,280.33,1.51,0.348,0.0292,0.278,0.89,7.72,-1.77,6.93,3.50,0.0451,0.0012,1.91,0.0482,0.0054,19415300.00,520369000.00,0.65,0.1463
IGTI3,2.38,313.89,1.25,4.814,0.011,0.663,3.90,10.43,-4.54,12.90,9.69,0.4614,-0.0019,2.11,0.0876,0.004,547704.00,4002910000.00,0.81,0.0775
IGTI11,17.32,326.32,1.30,5.005,0.0105,0.689,4.06,10.85,-4.72,13.31,10.01,0.4614,-0.0019,2.11,0.0876,0.004,67674900.00,4002910000.00,0.81,0.0775
